In [3]:
import tensorflow as tf
import numpy as np

In [49]:


NUM_CHANNELS = 1

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 28, 28, 1], name='X')
labels = tf.placeholder(tf.int32, [None, 1], name='labels')

input_layer = tf.reshape(X, [-1, 28, 28, 1])

patch_size = 4
input_patches = tf.extract_image_patches(input_layer,
  ksizes=[1, 4, 4, 1],
  strides=[1, 4, 4, 1],
  rates=[1, 1, 1, 1],
  padding='VALID'
)
input_patches_shape = tf.shape(input_patches)
input_patches = tf.reshape(input_patches, shape=(-1, tf.pow(input_patches_shape[1],2), patch_size, patch_size, 1))
input_patches_shape = tf.shape(input_patches)

batch_size = input_patches_shape[0]
patches_per_image = input_patches_shape[1]

new_shape = (input_patches_shape[0] * input_patches_shape[1], input_patches_shape[2], input_patches_shape[3], tf.constant(1))
input_patches = tf.reshape(input_patches, shape=(new_shape))

# Convolutional Layer #1
conv1 = tf.layers.conv2d(
  inputs=input_patches,
  filters=32,
  kernel_size=[5, 5],
  padding="same",
  activation=tf.nn.relu)

# Pooling Layer #1
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

# Convolutional Layer #2 and Pooling Layer #2
conv2 = tf.layers.conv2d(
  inputs=pool1,
  filters=64,
  kernel_size=[5, 5],
  padding="same",
  activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

# Dense Layer
pool2_flat = tf.reshape(pool2, [-1, 49, 1 * 1 * 64])
weights = tf.ones_like(pool2_flat)
pool2_flat = tf.einsum('ijk, ijk->ik', weights, pool2_flat)
dense = tf.layers.dense(inputs=pool2_flat, units=128, activation=tf.nn.relu)
dropout = tf.layers.dropout(
  inputs=dense, rate=0.4)
dropout = tf.reshape(dropout, shape=[-1, 128])
# dropout = tf.concat(dropout, axis=1)

# Logits Layer
logits = tf.layers.dense(inputs=dropout, units=10)

classes = tf.argmax(input=logits, axis=1)
probabilites = tf.nn.softmax(logits, name="softmax_tensor")

loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(
    loss=loss,
    global_step=tf.train.get_global_step()
)

_, accuracy = tf.metrics.accuracy(
          labels=labels, predictions=classes)

init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

In [45]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32).reshape(-1,1)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32).reshape(-1,1)

train_data = train_data.reshape(-1,28, 28, 1)
eval_data = eval_data.reshape(-1, 28 ,28, 1)

with tf.Session() as sess:
    sess.run(init_g)
    sess.run(init_l)
    
    weights_, pool2_flat_ = sess.run([weights, pool2_flat], feed_dict={
                                                            X: train_data[0:10],
                                                            labels: train_labels[0:10]
                                                        })

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [46]:
weights_.shape, pool2_flat_.shape

((10, 49, 64), (10, 64))

In [37]:
np.dot(weights_, pool2_flat_)

ValueError: shapes (10,49,64) and (10,49,64) not aligned: 64 (dim 2) != 49 (dim 1)

In [47]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32).reshape(-1,1)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32).reshape(-1,1)

train_data = train_data.reshape(-1,28, 28, 1)
eval_data = eval_data.reshape(-1, 28 ,28, 1)

total_epochs = 10
batch_size = 32
with tf.Session() as sess:
    sess.run(init_g)
    sess.run(init_l)
    
    for epoch_n in range(total_epochs):
        print("Epoch:", epoch_n)
        epoch_train_losses = []
        for b in range(0, train_data.shape[0], batch_size):
            batch_slice = slice(b,b+batch_size)
            _, loss_ = sess.run([train_op, loss], feed_dict={
                                                            X: train_data[batch_slice],
                                                            labels: train_labels[batch_slice]
                                                        })
            epoch_train_losses.append(loss_)
        
        train_loss_ = np.mean(epoch_train_losses)
        test_loss_, acc_, predictions_, labels_ = sess.run([loss, accuracy, classes, labels], feed_dict={
                                                X: eval_data,
                                                labels: eval_labels
                                            })
        print("Train loss:", train_loss_, "Test loss:", test_loss_, "Test acc:", acc_)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Epoch: 0
Train loss: 2.080736 Test loss: 1.9847587 Test acc: 0.2725
Epoch: 1
Train loss: 1.9331077 Test loss: 1.89262 Test acc: 0.2991
Epoch: 2
Train loss: 1.8224785 Test loss: 1.7493637 Test acc: 0.32863334
Epoch: 3
Train loss: 1.6655813 Test loss: 1.6010178 Test acc: 0.34775
Epoch: 4
Train loss: 1.5311625 Test loss: 1.520304 Test acc: 0.36234
Epoch: 5
Train loss: 1.4464592 Test loss: 1.4654785 Test acc: 0.37475
Epoch: 6
Train loss: 1.3890394 Test loss: 1.4153054 Test acc: 0.3866
Epoch: 7
Train loss: 1.3440602 Test loss: 1.3707612 Test acc: 0.398025
Epoch: 8
Train loss: 1.3062259 Test loss: 1.3284141 Test acc: 0.4087889
Epoch: 9


KeyboardInterrupt: 

In [ ]:
images.shape, patches.shape, which_patch_

In [26]:
train_data.reshape(-1, 28, 28, 1).shape

(55000, 28, 28, 1)

In [11]:


mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd1fc073208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [14]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 2.2995894, step = 0
INFO:tensorflow:global_step/sec: 213.864
INFO:tensorflow:loss = 2.2941153, step = 100 (0.468 sec)
INFO:tensorflow:global_step/sec: 232.189
INFO:tensorflow:loss = 2.276829, step = 200 (0.431 sec)
INFO:tensorflow:global_step/sec: 232.818
INFO:tensorflow:loss = 2.252899, step = 300 (0.429 sec)
INFO:tensorflow:global_step/sec: 232.112
INFO:tensorflow:loss = 2.2555707, step = 400 (0.431 sec)
INFO:tensorflow:global_step/sec: 232.137
INFO:tensorflow:loss = 2.2261717, step = 500 (0.431 sec)
INFO:tensorflow:global_step/sec: 230.018
INFO:tensorflow:loss = 2.189

INFO:tensorflow:loss = 0.24049634, step = 7900 (0.436 sec)
INFO:tensorflow:global_step/sec: 231.977
INFO:tensorflow:loss = 0.30147496, step = 8000 (0.431 sec)
INFO:tensorflow:global_step/sec: 232.204
INFO:tensorflow:loss = 0.22580679, step = 8100 (0.430 sec)
INFO:tensorflow:global_step/sec: 231.452
INFO:tensorflow:loss = 0.13308042, step = 8200 (0.432 sec)
INFO:tensorflow:global_step/sec: 233.57
INFO:tensorflow:loss = 0.2616243, step = 8300 (0.428 sec)
INFO:tensorflow:global_step/sec: 229.41
INFO:tensorflow:loss = 0.2373055, step = 8400 (0.436 sec)
INFO:tensorflow:global_step/sec: 233.181
INFO:tensorflow:loss = 0.17706211, step = 8500 (0.429 sec)
INFO:tensorflow:global_step/sec: 228.829
INFO:tensorflow:loss = 0.16255292, step = 8600 (0.437 sec)
INFO:tensorflow:global_step/sec: 233.297
INFO:tensorflow:loss = 0.14898622, step = 8700 (0.430 sec)
INFO:tensorflow:global_step/sec: 229.041
INFO:tensorflow:loss = 0.20537885, step = 8800 (0.435 sec)
INFO:tensorflow:global_step/sec: 232.281
INFO

INFO:tensorflow:global_step/sec: 229.653
INFO:tensorflow:loss = 0.050648645, step = 16100 (0.436 sec)
INFO:tensorflow:global_step/sec: 225.881
INFO:tensorflow:loss = 0.12430352, step = 16200 (0.443 sec)
INFO:tensorflow:global_step/sec: 231.591
INFO:tensorflow:loss = 0.089066856, step = 16300 (0.431 sec)
INFO:tensorflow:global_step/sec: 231.642
INFO:tensorflow:loss = 0.21386448, step = 16400 (0.432 sec)
INFO:tensorflow:global_step/sec: 229.239
INFO:tensorflow:loss = 0.101071164, step = 16500 (0.436 sec)
INFO:tensorflow:global_step/sec: 232.925
INFO:tensorflow:loss = 0.21071836, step = 16600 (0.431 sec)
INFO:tensorflow:global_step/sec: 229.959
INFO:tensorflow:loss = 0.18800846, step = 16700 (0.434 sec)
INFO:tensorflow:global_step/sec: 227.821
INFO:tensorflow:loss = 0.22449127, step = 16800 (0.440 sec)
INFO:tensorflow:global_step/sec: 225.251
INFO:tensorflow:loss = 0.21552356, step = 16900 (0.443 sec)
INFO:tensorflow:global_step/sec: 233.279
INFO:tensorflow:loss = 0.20754051, step = 17000

In [15]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-04-11:54:36
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-04-11:54:37
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9693, global_step = 20000, loss = 0.10208161
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: /tmp/mnist_convnet_model/model.ckpt-20000
{'accuracy': 0.9693, 'loss': 0.10208161, 'global_step': 20000}
